In [1]:
import os
from os.path import join

import numpy as np
import pandas as pd

In [2]:
DATA_PATH = join("..", "data", "Dyslexia_1_ready_data_fixations.csv")

In [3]:
def read_dataset(path: str = DATA_PATH):
    df = pd.read_csv(path, index_col="Unnamed: 0")
    
    df["SentenceID"] = df["Sentence_ID"]
    df["x"] = df["FIX_X"].apply(lambda x: float(x.replace(",", ".")))
    df["y"] = df["FIX_Y"].apply(lambda x: float(x.replace(",", ".")))
    df.x /= df.x.max()
    df.y /= df.y.max()
    df["duration"] = df["FIX_DURATION"].apply(lambda x: float(x) / 1000)
    df["timestamp"] = df.duration.cumsum()
    df["AOI_1"] = np.random.choice([0, 1], size=len(df))
    df["AOI_2"] = np.random.choice([11, 22, 33], size=len(df))
    df["AOI_3"] = np.random.choice(['A', 'B', 'C'], size=len(df))
    
    df.drop(["Word_Number", "IQ", "FIX_X", "FIX_Y", "FIX_DURATION", "Age", "Sentence_ID"], axis=1, inplace=True)
    return df

In [4]:
df = read_dataset()
df.head()

,SubjectID,Group,Sex,SentenceID,x,y,duration,timestamp,AOI_1,AOI_2,AOI_3
0,nnr8,3,fem,27,0.101158,0.726088,0.345,0.345,1,22,A
1,nnr8,3,fem,27,0.102406,0.754083,0.221,0.566,1,22,B
2,nnr8,3,fem,27,0.126049,0.755455,0.204,0.770,1,11,C
3,nnr8,3,fem,27,0.142134,0.761356,0.289,1.059,1,11,C
4,nnr8,3,fem,27,0.109963,0.766433,0.600,1.659,1,22,A


### 0. `Extractor` над нормальным датасетом.

In [5]:
import eyefeatures.features.stats as eye_stats
from eyefeatures.features.extractor import Extractor

sac_feats_stats = {
    'length': ['min', 'max'],
    'acceleration': ['mean']
}

sac_feats_stats_shift = {
    'acceleration': ['mean']
}

sf = eye_stats.SaccadeFeatures(
    features_stats=sac_feats_stats,
    shift_features=sac_feats_stats_shift,
    shift_pk=['SentenceID']
)

extractor = Extractor(
    features=[sf],
    x='x',
    y='y',
    t='timestamp',
    duration='duration',
    # dispersion='dispersion',
    # path_pk=['SubjectID'],
    pk=['SubjectID', 'SentenceID'],
    return_df=True
)

extractor.fit_transform(df).head()

100%|██████████| 1/1 [00:00<00:00,  1.93it/s]


,sac_length_min,sac_length_max,sac_acceleration_mean,sac_acceleration_mean_shift_SentenceID
nnr26_4,0.019488,0.092498,0.509739,-15.571444
nnr26_5,0.013345,0.080962,0.459010,-10.890528
nnr26_6,0.018656,0.112882,0.578460,-14.121530
nnr26_7,0.015868,0.118729,0.386805,-15.961155
nnr26_8,0.023546,0.090291,0.434029,-10.641588


### 1. Warnings & Errors.

В `BaseTransformer`, `Extractor`, и `StatsTransformer` добавлен параметр `warn: bool = True`, который контролирует показ всех `warning`-ов.

#### 1.1. Errors.

##### 1.1.1. Обработка `NaN`. Ожидается: warning в случае `NaN` в данных, `ValueError` в случае `NaN` в `pk`/`shift_pk`.

In [6]:
dfn = df.copy()
dfn.loc[0, "x"] = None
dfn.loc[0, "y"] = np.nan
extractor.fit_transform(dfn).head()

/tmp/ipykernel_15875/889556869.py:4: UserWarning: Group nnr8_27 has missing values. Dropping them.
  extractor.fit_transform(dfn).head()
100%|██████████| 1/1 [00:00<00:00,  1.84it/s]


,sac_length_min,sac_length_max,sac_acceleration_mean,sac_acceleration_mean_shift_SentenceID
nnr26_4,0.019488,0.092498,0.509739,-15.571444
nnr26_5,0.013345,0.080962,0.459010,-10.890528
nnr26_6,0.018656,0.112882,0.578460,-14.121530
nnr26_7,0.015868,0.118729,0.386805,-15.961155
nnr26_8,0.023546,0.090291,0.434029,-10.641588


In [7]:
dfn.loc[0, "SubjectID"] = None
try:
    extractor.fit_transform(dfn).head()
except ValueError as e:
    print(e)
    print("OK")

Found missing values in pk.
OK


#### 1.2. Warnings.

##### 1.2.1. Отсутствие warning-а при `warn=False`.

In [8]:
extractor_no_warn = Extractor(
    features=[sf],
    x='x',
    y='y',
    t='timestamp',
    duration='duration',
    # dispersion='dispersion',
    # path_pk=['SubjectID'],
    pk=['SubjectID', 'SentenceID'],
    return_df=True,
    warn=False
)

dfn = df.copy()
dfn.loc[0, "x"] = None
dfn.loc[0, "y"] = np.nan

In [9]:
extractor.fit_transform(dfn).head();

/tmp/ipykernel_15875/1683100184.py:1: UserWarning: Group nnr8_27 has missing values. Dropping them.
  extractor.fit_transform(dfn).head();
100%|██████████| 1/1 [00:00<00:00,  1.97it/s]


In [10]:
extractor_no_warn.fit_transform(dfn).head();

100%|██████████| 1/1 [00:00<00:00,  1.99it/s]


##### 1.2.2. Неизвестные значения в `shift_pk`. Ожидается: warning.

In [11]:
sf = eye_stats.SaccadeFeatures(
    features_stats=sac_feats_stats,
    shift_features=sac_feats_stats_shift,
    shift_pk=['SentenceID']
)

extractor = Extractor(
    features=[sf],
    x='x',
    y='y',
    t='timestamp',
    duration='duration',
    # dispersion='dispersion',
    # path_pk=['SubjectID'],
    pk=['SubjectID', 'SentenceID'],
    return_df=True
)

In [12]:
dfn = df.copy()
dfn.loc[0, "SentenceID"] = -1

extractor.fit(df)
extractor.transform(dfn)

  0%|          | 0/1 [00:00<?, ?it/s]/home/vagiz/.cache/pypoetry/virtualenvs/eyefeatures-dVI54DNx-py3.12/lib/python3.12/site-packages/sklearn/utils/_set_output.py:313: UserWarning: Group -1 for shift_pk SentenceID was not seen during `fit`.Average across all values of SentenceID is used instead.
  data_to_wrap = f(self, X, *args, **kwargs)
100%|██████████| 1/1 [00:00<00:00,  1.91it/s]


,sac_length_min,sac_length_max,sac_acceleration_mean,sac_acceleration_mean_shift_SentenceID
nnr26_4,0.019488,0.092498,0.509739,-15.571444
nnr26_5,0.013345,0.080962,0.459010,-10.890528
nnr26_6,0.018656,0.112882,0.578460,-14.121530
nnr26_7,0.015868,0.118729,0.386805,-15.961155
nnr26_8,0.023546,0.090291,0.434029,-10.641588
...,...,...,...,...
nnr85_29,0.014600,0.140295,0.595447,-14.921391
nnr85_30,0.014928,0.417827,1.273437,-11.274076
nnr85_31,0.034824,0.161574,0.607337,-14.189839
nnr85_32,0.013381,0.140145,0.725873,-10.295705


### 2. Несколько `shift_pk` и проверка, что можно не подмножество `pk`.

#### 2.0. Дефолтные случаи.

##### 2.0.0. Дефолтный случай `pk=None`, `shift_features=None`. Ожидается: одна строка и 3 фичи.

In [13]:
sf = eye_stats.SaccadeFeatures(
    features_stats=sac_feats_stats,
    # shift_features=sac_feats_stats_shift,
    # shift_pk=['SentenceID']
)

extractor = Extractor(
    features=[sf],
    x='x',
    y='y',
    t='timestamp',
    duration='duration',
    # dispersion='dispersion',
    # path_pk=['SubjectID'],
    # pk=['SubjectID'],
    return_df=True
)

extractor.fit_transform(df).head()

100%|██████████| 1/1 [00:00<00:00, 694.88it/s]


,sac_length_min,sac_length_max,sac_acceleration_mean
0,0.000882,0.891518,10.213596


##### 2.0.1. Дефолтный случай `pk=...`, `shift_features=None`. Ожидается: 3 фичи.

In [14]:
sf = eye_stats.SaccadeFeatures(
    features_stats=sac_feats_stats,
    # shift_features=sac_feats_stats_shift,
    # shift_pk=['SentenceID']
)

extractor = Extractor(
    features=[sf],
    x='x',
    y='y',
    t='timestamp',
    duration='duration',
    # dispersion='dispersion',
    # path_pk=['SubjectID'],
    pk=['SubjectID', 'SentenceID'],
    return_df=True
)

extractor.fit_transform(df).head()

100%|██████████| 1/1 [00:00<00:00,  2.84it/s]


,sac_length_min,sac_length_max,sac_acceleration_mean
nnr26_4,0.019488,0.092498,0.509739
nnr26_5,0.013345,0.080962,0.459010
nnr26_6,0.018656,0.112882,0.578460
nnr26_7,0.015868,0.118729,0.386805
nnr26_8,0.023546,0.090291,0.434029


#### 2.1. Можно `shift_pk` не подмножество `shift_features`.

##### 2.1.1. Можно `shift_pk` не подмножество `shift_features`. Ожидается: 3 фичи + 1 shift фича.

In [15]:
sf = eye_stats.SaccadeFeatures(
    features_stats=sac_feats_stats,
    shift_features=sac_feats_stats_shift,
    shift_pk=['SentenceID']
)

extractor = Extractor(
    features=[sf],
    x='x',
    y='y',
    t='timestamp',
    duration='duration',
    # dispersion='dispersion',
    # path_pk=['SubjectID'],
    pk=['SubjectID'],
    return_df=True
)

extractor.fit_transform(df).head()

100%|██████████| 1/1 [00:00<00:00, 24.41it/s]


,sac_length_min,sac_length_max,sac_acceleration_mean,sac_acceleration_mean_shift_SentenceID
nnr26,0.006898,0.879102,0.739731,-90.512813
nnr27,0.007935,0.868788,0.504197,-56.421544
nnr33,0.002138,0.866240,6.324986,-37.798959
nnr36,0.001416,0.784169,0.487095,-83.633605
nnr41,0.002338,0.857037,150.445112,53.017588


#### 2.2. Комбинации `shift_pk` и `shift_features`.

##### 2.2.1. Можно несколько `shift_pk` на один `shift_features`. Ожидается: 3 фичи + 3 shift фичи.

In [16]:
sf = eye_stats.SaccadeFeatures(
    features_stats=sac_feats_stats,
    shift_features=sac_feats_stats_shift,
    shift_pk=(['SentenceID', 'SubjectID'], ['SentenceID'], ['SubjectID'])
)

extractor = Extractor(
    features=[sf],
    x='x',
    y='y',
    t='timestamp',
    duration='duration',
    # dispersion='dispersion',
    # path_pk=['SubjectID'],
    pk=['SubjectID'],
    return_df=True
)

extractor.fit_transform(df).head()

100%|██████████| 1/1 [00:00<00:00, 13.70it/s]


,sac_length_min,sac_length_max,sac_acceleration_mean,sac_acceleration_mean_shift_SentenceID_SubjectID,sac_acceleration_mean_shift_SentenceID,sac_acceleration_mean_shift_SubjectID
nnr26,0.006898,0.879102,0.739731,0.172832,-90.512813,0.0
nnr27,0.007935,0.868788,0.504197,0.158958,-56.421544,0.0
nnr33,0.002138,0.866240,6.324986,5.701891,-37.798959,0.0
nnr36,0.001416,0.784169,0.487095,0.098271,-83.633605,0.0
nnr41,0.002338,0.857037,150.445112,149.332152,53.017588,0.0


##### 2.2.2. Можно несколько `shift_pk` и несколько `shift_features`. Ожидается: 3 фичи + 3 shift фичи.

In [17]:
sf = eye_stats.SaccadeFeatures(
    features_stats={'length': ['min', 'max', 'median']},
    shift_features=({'length': ['min']}, {'length': ['max', 'median']}),
    shift_pk=(['SentenceID', 'SubjectID'], ['SentenceID'])
)

extractor = Extractor(
    features=[sf],
    x='x',
    y='y',
    t='timestamp',
    duration='duration',
    # dispersion='dispersion',
    # path_pk=['SubjectID'],
    pk=['SubjectID'],
    return_df=True
)

extractor.fit_transform(df).head()

/home/vagiz/.cache/pypoetry/virtualenvs/eyefeatures-dVI54DNx-py3.12/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/vagiz/.cache/pypoetry/virtualenvs/eyefeatures-dVI54DNx-py3.12/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/vagiz/.cache/pypoetry/virtualenvs/eyefeatures-dVI54DNx-py3.12/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/vagiz/.cache/pypoetry/virtualenvs/eyefeatures-dVI54DNx-py3.12/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/vagiz/.cache/pypoetry/virtualenvs/eyefeatures-dVI54DNx-py3.12/lib/python3.12/site-packages/numpy/lib/nanfu

,sac_length_min,sac_length_max,sac_length_median,sac_length_min_shift_SentenceID_SubjectID,sac_length_max_shift_SentenceID,sac_length_median_shift_SentenceID
nnr26,0.006898,0.879102,0.043258,-0.008151,-2.928285,-0.899397
nnr27,0.007935,0.868788,0.042489,-0.005300,-2.933991,-0.900745
nnr33,0.002138,0.866240,0.037203,-0.010555,-2.981443,-0.894471
nnr36,0.001416,0.784169,0.025004,-0.007431,-3.039331,-0.906430
nnr41,0.002338,0.857037,0.049938,-0.011231,-2.976189,-0.893923


### 3. Можно несколько разных `AOI` колонок.

#### 3.0. Дефолтные случаи.

##### 3.0.1. `aoi=None`. Ожидается: 3 фичи + 2 shift фичи.

In [18]:
sf = eye_stats.SaccadeFeatures(
    features_stats=sac_feats_stats,
    shift_features=sac_feats_stats_shift,
    shift_pk=(['SentenceID'], ['SubjectID'])
)

extractor = Extractor(
    features=[sf],
    x='x',
    y='y',
    t='timestamp',
    duration='duration',
    # dispersion='dispersion',
    # path_pk=['SubjectID'],
    aoi=None,
    pk=['SubjectID'],
    return_df=True
)

extractor.fit_transform(df).head()

100%|██████████| 1/1 [00:00<00:00, 17.99it/s]


,sac_length_min,sac_length_max,sac_acceleration_mean,sac_acceleration_mean_shift_SentenceID,sac_acceleration_mean_shift_SubjectID
nnr26,0.006898,0.879102,0.739731,-90.512813,0.0
nnr27,0.007935,0.868788,0.504197,-56.421544,0.0
nnr33,0.002138,0.866240,6.324986,-37.798959,0.0
nnr36,0.001416,0.784169,0.487095,-83.633605,0.0
nnr41,0.002338,0.857037,150.445112,53.017588,0.0


#### 3.1. Одно и более значений `AOI` + `calc_without_aoi`.

##### 3.1.1. Одно значение `AOI`. Ожидается: (3 фичи + 2 shift фичи) x 2 для `AOI_1=0` и `AOI_1=1`.

In [19]:
sf = eye_stats.SaccadeFeatures(
    features_stats=sac_feats_stats,
    shift_features=sac_feats_stats_shift,
    shift_pk=(['SentenceID'], ['SubjectID'])
)

extractor = Extractor(
    features=[sf],
    x='x',
    y='y',
    t='timestamp',
    duration='duration',
    # dispersion='dispersion',
    # path_pk=['SubjectID'],
    aoi='AOI_1',
    pk=['SubjectID'],
    return_df=True
)

extractor.fit_transform(df).head()

100%|██████████| 1/1 [00:00<00:00,  9.20it/s]


,sac_length_AOI_1[1]_min,sac_length_AOI_1[1]_max,sac_acceleration_AOI_1[1]_mean,sac_acceleration_AOI_1[1]_mean_shift_SentenceID,sac_acceleration_AOI_1[1]_mean_shift_SubjectID,sac_length_AOI_1[0]_min,sac_length_AOI_1[0]_max,sac_acceleration_AOI_1[0]_mean,sac_acceleration_AOI_1[0]_mean_shift_SentenceID,sac_acceleration_AOI_1[0]_mean_shift_SubjectID
nnr26,0.011173,0.716501,0.606357,-265.692016,0.0,0.007243,0.767293,0.756903,-13.923686,0.0
nnr27,0.007935,0.868788,0.614128,-151.055144,0.0,0.012107,0.705549,0.365033,-14.079203,0.0
nnr33,0.012854,0.787394,22.865037,-86.721009,0.0,0.004336,0.866240,1.012183,-13.488985,0.0
nnr36,0.003097,0.784169,0.644782,-242.874640,0.0,0.004829,0.716495,0.377694,-13.661892,0.0
nnr41,0.004400,0.702948,13.674205,-272.888185,0.0,0.006442,0.857037,607.626732,592.592756,0.0


##### 3.1.2. Одно значение `AOI`. Ожидается: (3 фичи + 2 shift фичи) x (1 + 2) для `calc_with_aoi=True`, `AOI_1=0` и `AOI_1=1`.

In [20]:
sf = eye_stats.SaccadeFeatures(
    features_stats=sac_feats_stats,
    shift_features=sac_feats_stats_shift,
    shift_pk=(['SentenceID'], ['SubjectID']),
    calc_without_aoi=True
)

extractor = Extractor(
    features=[sf],
    x='x',
    y='y',
    t='timestamp',
    duration='duration',
    # dispersion='dispersion',
    # path_pk=['SubjectID'],
    aoi='AOI_1',
    pk=['SubjectID'],
    return_df=True
)

extractor.fit_transform(df).head()

100%|██████████| 1/1 [00:00<00:00,  5.94it/s]


,sac_length_min,sac_length_max,sac_acceleration_mean,sac_acceleration_mean_shift_SentenceID,sac_acceleration_mean_shift_SubjectID,sac_length_AOI_1[1]_min,sac_length_AOI_1[1]_max,sac_acceleration_AOI_1[1]_mean,sac_acceleration_AOI_1[1]_mean_shift_SentenceID,sac_acceleration_AOI_1[1]_mean_shift_SubjectID,sac_length_AOI_1[0]_min,sac_length_AOI_1[0]_max,sac_acceleration_AOI_1[0]_mean,sac_acceleration_AOI_1[0]_mean_shift_SentenceID,sac_acceleration_AOI_1[0]_mean_shift_SubjectID
nnr26,0.006898,0.879102,0.739731,-90.512813,0.0,0.011173,0.716501,0.606357,-265.692016,0.0,0.007243,0.767293,0.756903,-13.923686,0.0
nnr27,0.007935,0.868788,0.504197,-56.421544,0.0,0.007935,0.868788,0.614128,-151.055144,0.0,0.012107,0.705549,0.365033,-14.079203,0.0
nnr33,0.002138,0.866240,6.324986,-37.798959,0.0,0.012854,0.787394,22.865037,-86.721009,0.0,0.004336,0.866240,1.012183,-13.488985,0.0
nnr36,0.001416,0.784169,0.487095,-83.633605,0.0,0.003097,0.784169,0.644782,-242.874640,0.0,0.004829,0.716495,0.377694,-13.661892,0.0
nnr41,0.002338,0.857037,150.445112,53.017588,0.0,0.004400,0.702948,13.674205,-272.888185,0.0,0.006442,0.857037,607.626732,592.592756,0.0


##### 3.1.3. Три значения `AOI`. Ожидается: (1 фича + 2 shift фичи) x (2 + 3 + 3) = 24 для `AOI_1` $\in [0, 1]$, `AOI_2` $\in [11, 22, 33]$, `AOI_3` $\in [A, B, C]$.


In [21]:
sf = eye_stats.SaccadeFeatures(
    features_stats={'length': ['min']},
    shift_features={'length': ['min']},
    shift_pk=(['SentenceID'], ['SubjectID'])
)

extractor = Extractor(
    features=[sf],
    x='x',
    y='y',
    t='timestamp',
    duration='duration',
    # dispersion='dispersion',
    # path_pk=['SubjectID'],
    aoi=['AOI_1', 'AOI_2', 'AOI_3'],
    pk=['SubjectID'],
    return_df=True
)

extractor.fit_transform(df).head()

100%|██████████| 1/1 [00:00<00:00,  2.61it/s]


,sac_length_AOI_1[1]_min,sac_length_AOI_1[1]_min_shift_SentenceID,sac_length_AOI_1[1]_min_shift_SubjectID,sac_length_AOI_1[0]_min,sac_length_AOI_1[0]_min_shift_SentenceID,sac_length_AOI_1[0]_min_shift_SubjectID,sac_length_AOI_2[22]_min,sac_length_AOI_2[22]_min_shift_SentenceID,sac_length_AOI_2[22]_min_shift_SubjectID,sac_length_AOI_2[11]_min,...,sac_length_AOI_2[33]_min_shift_SubjectID,sac_length_AOI_3[A]_min,sac_length_AOI_3[A]_min_shift_SentenceID,sac_length_AOI_3[A]_min_shift_SubjectID,sac_length_AOI_3[B]_min,sac_length_AOI_3[B]_min_shift_SentenceID,sac_length_AOI_3[B]_min_shift_SubjectID,sac_length_AOI_3[C]_min,sac_length_AOI_3[C]_min_shift_SentenceID,sac_length_AOI_3[C]_min_shift_SubjectID
nnr26,0.011173,-0.515680,0.0,0.007243,-0.513577,0.0,0.013875,-0.588642,0.0,0.009377,...,0.0,0.010186,-0.595604,0.0,0.011133,-0.619200,0.0,0.013601,-0.582428,0.0
nnr27,0.007935,-0.501647,0.0,0.012107,-0.509164,0.0,0.013915,-0.598368,0.0,0.010210,...,0.0,0.013915,-0.593497,0.0,0.010569,-0.618051,0.0,0.010535,-0.560018,0.0
nnr33,0.012854,-0.493201,0.0,0.004336,-0.500981,0.0,0.012854,-0.586866,0.0,0.008263,...,0.0,0.009477,-0.593291,0.0,0.009666,-0.604146,0.0,0.008263,-0.577650,0.0
nnr36,0.003097,-0.518631,0.0,0.004829,-0.501101,0.0,0.005707,-0.591722,0.0,0.008489,...,0.0,0.004243,-0.599212,0.0,0.004899,-0.618931,0.0,0.001416,-0.578460,0.0
nnr41,0.004400,-0.525186,0.0,0.006442,-0.515637,0.0,0.014302,-0.591852,0.0,0.005504,...,0.0,0.002338,-0.587699,0.0,0.012672,-0.606514,0.0,0.006884,-0.576902,0.0


##### 3.1.4. Три значения `AOI`. Ожидается: (1 фича + 2 shift фичи) x (2 + 3 + 3 + 1) = 27 для `AOI_1` $\in [0, 1]$, `AOI_2` $\in [11, 22, 33]$, `AOI_3` $\in [A, B, C]$, `calc_without_aoi=True`.


In [22]:
sf = eye_stats.SaccadeFeatures(
    features_stats={'length': ['min']},
    shift_features={'length': ['min']},
    shift_pk=(['SentenceID'], ['SubjectID']),
    calc_without_aoi=True
)

extractor = Extractor(
    features=[sf],
    x='x',
    y='y',
    t='timestamp',
    duration='duration',
    # dispersion='dispersion',
    # path_pk=['SubjectID'],
    aoi=['AOI_1', 'AOI_2', 'AOI_3'],
    pk=['SubjectID'],
    return_df=True
)

extractor.fit_transform(df).head()

100%|██████████| 1/1 [00:00<00:00,  2.42it/s]


,sac_length_min,sac_length_min_shift_SentenceID,sac_length_min_shift_SubjectID,sac_length_AOI_1[1]_min,sac_length_AOI_1[1]_min_shift_SentenceID,sac_length_AOI_1[1]_min_shift_SubjectID,sac_length_AOI_1[0]_min,sac_length_AOI_1[0]_min_shift_SentenceID,sac_length_AOI_1[0]_min_shift_SubjectID,sac_length_AOI_2[22]_min,...,sac_length_AOI_2[33]_min_shift_SubjectID,sac_length_AOI_3[A]_min,sac_length_AOI_3[A]_min_shift_SentenceID,sac_length_AOI_3[A]_min_shift_SubjectID,sac_length_AOI_3[B]_min,sac_length_AOI_3[B]_min_shift_SentenceID,sac_length_AOI_3[B]_min_shift_SubjectID,sac_length_AOI_3[C]_min,sac_length_AOI_3[C]_min_shift_SentenceID,sac_length_AOI_3[C]_min_shift_SubjectID
nnr26,0.006898,-0.299538,0.0,0.011173,-0.515680,0.0,0.007243,-0.513577,0.0,0.013875,...,0.0,0.010186,-0.595604,0.0,0.011133,-0.619200,0.0,0.013601,-0.582428,0.0
nnr27,0.007935,-0.298883,0.0,0.007935,-0.501647,0.0,0.012107,-0.509164,0.0,0.013915,...,0.0,0.013915,-0.593497,0.0,0.010569,-0.618051,0.0,0.010535,-0.560018,0.0
nnr33,0.002138,-0.298456,0.0,0.012854,-0.493201,0.0,0.004336,-0.500981,0.0,0.012854,...,0.0,0.009477,-0.593291,0.0,0.009666,-0.604146,0.0,0.008263,-0.577650,0.0
nnr36,0.001416,-0.299904,0.0,0.003097,-0.518631,0.0,0.004829,-0.501101,0.0,0.005707,...,0.0,0.004243,-0.599212,0.0,0.004899,-0.618931,0.0,0.001416,-0.578460,0.0
nnr41,0.002338,-0.302846,0.0,0.004400,-0.525186,0.0,0.006442,-0.515637,0.0,0.014302,...,0.0,0.002338,-0.587699,0.0,0.012672,-0.606514,0.0,0.006884,-0.576902,0.0


#### Проверка названий фичей

In [23]:
extractor.features[0].feature_names_in_

['sac_length_min',
 'sac_length_min_shift_SentenceID',
 'sac_length_min_shift_SubjectID',
 'sac_length_AOI_1[1]_min',
 'sac_length_AOI_1[1]_min_shift_SentenceID',
 'sac_length_AOI_1[1]_min_shift_SubjectID',
 'sac_length_AOI_1[0]_min',
 'sac_length_AOI_1[0]_min_shift_SentenceID',
 'sac_length_AOI_1[0]_min_shift_SubjectID',
 'sac_length_AOI_2[22]_min',
 'sac_length_AOI_2[22]_min_shift_SentenceID',
 'sac_length_AOI_2[22]_min_shift_SubjectID',
 'sac_length_AOI_2[11]_min',
 'sac_length_AOI_2[11]_min_shift_SentenceID',
 'sac_length_AOI_2[11]_min_shift_SubjectID',
 'sac_length_AOI_2[33]_min',
 'sac_length_AOI_2[33]_min_shift_SentenceID',
 'sac_length_AOI_2[33]_min_shift_SubjectID',
 'sac_length_AOI_3[A]_min',
 'sac_length_AOI_3[A]_min_shift_SentenceID',
 'sac_length_AOI_3[A]_min_shift_SubjectID',
 'sac_length_AOI_3[B]_min',
 'sac_length_AOI_3[B]_min_shift_SentenceID',
 'sac_length_AOI_3[B]_min_shift_SubjectID',
 'sac_length_AOI_3[C]_min',
 'sac_length_AOI_3[C]_min_shift_SentenceID',
 'sac_len